# Contract risk analysis

In [5]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();

var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");

record Car (string make, string model, string basemodel, string color, int year, string condition, int mileage, int price);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

### Process a completion using the Azure SDK

In [6]:
async Task<string> process_competion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

### Render an Scriban template

In [7]:
string render_tempate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

In [13]:
string ReadAllFiles(string path) {
    var files = System.IO.Directory.GetFiles(path);
    var sb = new StringBuilder();
    foreach (var file in files) {
        sb.Append(System.IO.File.ReadAllText(file));
    }
    return sb.ToString();
}
var content = ReadAllFiles("data/");

In [14]:
//# Create a semantic kernel inline function
var faq_template = @"System:
You are an agent that can help generate FAQ documents.
Today's date is {{date}}.

User:
For the following text, generate a detailed FAQ document titled 'Contoso Company Benefits FAQ'. 

Text: ===
{{input}}
===

Sample Output format: ===
Title: // title
Date: // Current date

Q: What is 401k?
A:  A: A 401k plan is a retirement savings plan that allows you to contribute a portion of your pre-tax income to a designated account.
===

Use only the provided text. Do not use any external sources.";

var datestr = DateTime.Now.ToString("yyyy-MM-dd HH:mm:ss");
var prompt = render_tempate(faq_template, new {input=content, date=datestr});
Console.WriteLine(prompt);

System:
You are an agent that can help generate FAQ documents.
Today's date is 2024-03-07 11:39:07.

User:
For the following text, generate a detailed FAQ document titled 'Contoso Company Benefits FAQ'. 

Text: ===
401K Policy
Contoso

A 401k plan is a retirement savings plan that allows you to contribute a portion of your pre-tax income to a designated account. The company offers a 401k plan to all eligible employees as part of its benefits package. The company also matches your contributions up to a certain percentage, depending on your years of service and salary level. This means that the company will add money to your 401k account for every dollar you contribute, up to the matching limit.

The 401k plan is administered by a third-party provider, who is responsible for investing your contributions and providing you with online access to your account. You can choose from a variety of investment options, ranging from conservative to aggressive, depending on your risk tolerance and re

### Proces Prompt and Completion

In [15]:
// Execute the SK function
Console.WriteLine(await process_competion(prompt, 1000));

Title: Contoso Company Benefits FAQ
Date: 2024-03-07

Q: What is a 401k plan?
A: A 401k plan is a retirement savings plan that allows you to contribute a portion of your pre-tax income to a designated account. It is part of Contoso's benefits package offered to all eligible employees.

Q: Does Contoso offer any contribution matching for the 401k plan?
A: Yes, Contoso matches your contributions up to a certain percentage, which is determined by your years of service and salary level.

Q: Who administers the 401k plan for Contoso?
A: The 401k plan is administered by a third-party provider responsible for investing your contributions and providing online access to your account.

Q: Can I choose how to invest my 401k contributions?
A: Yes, you can choose from a variety of investment options, ranging from conservative to aggressive, based on your risk tolerance and retirement goals. You can also change your contribution amount and investment allocation at any time, with certain restrictions